## Desafio 1 - Leave-One-Instance-Out Bias-Free Protocol

In [13]:
# Artifício para calcular tempo total do notebook Jupyter
from datetime import datetime 
start_time = datetime.now()

import pandas as pd
import numpy as np
import seaborn as sns
import logging
import warnings
import sys
from math import ceil
from matplotlib import pyplot as plt
from time import time
from pathlib import Path
from tsfresh.feature_extraction import extract_features
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.feature_extraction import MinimalFCParameters
from sklearn.svm import OneClassSVM
from sklearn.covariance import EllipticEnvelope
from sklearn.neighbors import LocalOutlierFactor
from sklearn.ensemble import IsolationForest
from sklearn.dummy import DummyClassifier
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import ParameterGrid
import re

logging.getLogger('tsfresh').setLevel(logging.ERROR)
warnings.simplefilter(action='ignore')

%matplotlib inline
%config InlineBackend.figure_format = 'png'

data_path = Path('./', 'data')
random_state = 1
np.random.seed(random_state)
events_names = {0: 'Normal',
                1: 'Aumento Abrupto de BSW',
                2: 'Fechamento Espúrio de DHSV',
                3: 'Intermitência Severa',
                4: 'Instabilidade de Fluxo',
                5: 'Perda Rápida de Produtividade',
                6: 'Restrição Rápida em CKP',
                7: 'Incrustação em CKP',
                8: 'Hidrato em Linha de Produção'
               }
vars = ['P-PDG',
        'P-TPT',
        'T-TPT',
        'P-MON-CKP',
        'T-JUS-CKP',
        'P-JUS-CKGL',
        'T-JUS-CKGL',
        'QGL']
columns = ['timestamp'] + vars + ['class'] 
normal_class_code = 0
abnormal_classes_codes = [1, 2, 5, 6, 7, 8]
sample_size = 3*60              # Nas observações = segundos
min_normal_period_size = 20*60  # Nas observações = segundos
split_range = 0.6               # Porcentagem de separação entre treino/teste
max_samples_per_period = 15     # limitação por 'segurança'
df_fc_p = MinimalFCParameters() # Ver documentação da biblioteca tsfresh
df_fc_p.pop('sum_values')       # Remove feature inapropriada
df_fc_p.pop('length')           # Remove feature inapropriada
max_nan_percent = 0.1           # Para seleção de variáveis úteis
std_vars_min = 0.01             # Para seleção de variáveis úteis
clfs = {}                       # Dicionário para lista de classificadores a serem experimentados
disable_progressbar = True      # Para menos saídas no notebook

def extract_well_number(filename):
    """Extrai o número do poço do nome do arquivo."""
    # Padrão: WELL-00017_20140317151743.csv -> retorna 17
    match = re.search(r'WELL-(\d+)', str(filename))
    if match:
        return int(match.group(1))
    return None

def class_and_file_generator(data_path, real=False, simulated=False, drawn=False):
    """Gerador de lista contendo número da classe e caminho do arquivo de acordo com a fonte da instância."""    
    for class_path in data_path.iterdir():
        if class_path.is_dir():
            class_code = int(class_path.stem)
            for instance_path in class_path.iterdir():
                if (instance_path.suffix == '.csv'):
                    if (simulated and instance_path.stem.startswith('SIMULATED')) or \
                       (drawn and instance_path.stem.startswith('DRAWN')) or \
                       (real and (not instance_path.stem.startswith('SIMULATED')) and \
                       (not instance_path.stem.startswith('DRAWN'))):
                        yield class_code, instance_path

def load_instance(instance_path):
    """Função que carrega cada instância individualmente"""
    try:
        well, instance_id = instance_path.stem.split('_')
        df = pd.read_csv(instance_path, sep=',', header=0)
        assert (df.columns == columns).all(), \
            f'Colunas inválidas no arquivo {str(instance_path)}: {str(df.columns.tolist())}'
        return df
    except Exception as e:
        raise Exception(f'Erro ao ler arquivo {instance_path}: {e}')

def extract_samples_for_test(df, class_code):
    """
    Extrai apenas as amostras de TESTE de um arquivo:
    - 40% das amostras normais
    - 100% das amostras de falha (transitório e em regime)
    """
    # Obtém os rótulos das observações e seu conjunto inequívoco
    ols = list(df['class'])
    set_ols = set()
    for ol in ols:
        if ol in set_ols or np.isnan(ol):
            continue
        set_ols.add(int(ol))       
    
    # Descarta os rótulos das observações e substitui todos os nan por 0
    df_vars = df.drop('class', axis=1).fillna(0)  
    
    # Inicializa objetos que serão retornados
    df_samples_test = pd.DataFrame()
    y_test = []
    sample_id = 0
    
    # Processa período normal - apenas para TESTE (40%)
    if normal_class_code in set_ols:
        f_idx = ols.index(normal_class_code)
        l_idx = len(ols)-1-ols[::-1].index(normal_class_code)
        
        max_samples_normal = l_idx-f_idx+1-sample_size
        if max_samples_normal > 0:
            num_normal_samples = min(max_samples_per_period, max_samples_normal)
            # Apenas 40% para teste
            num_test_samples = num_normal_samples - int(split_range * num_normal_samples)
            
            if num_test_samples > 0:
                if num_normal_samples == max_samples_normal:
                    step_max = 1
                else:
                    step_max = (max_samples_normal-1) // (max_samples_per_period-1)
                step_wanted = sample_size
                step = min(step_wanted, step_max)
                
                num_train_samples = int(split_range * num_normal_samples)
                for idx in range(num_train_samples, num_train_samples + num_test_samples):
                    f_idx_c = l_idx-sample_size+1-(num_normal_samples-1-idx)*step
                    l_idx_c = f_idx_c+sample_size
                    df_sample = df_vars.iloc[f_idx_c:l_idx_c, :].copy()
                    df_sample.insert(loc=0, column='id', value=sample_id)
                    df_samples_test = df_samples_test.append(df_sample, ignore_index=True)
                    y_test.append(normal_class_code)
                    sample_id += 1
    
    # Processa período transitório - 100% para TESTE
    transient_code = class_code + 100
    if transient_code in set_ols:
        f_idx = ols.index(transient_code)
        if f_idx-(sample_size-1) > 0:
            f_idx = f_idx-(sample_size-1)
        else:
            f_idx = 0
        l_idx = len(ols)-1-ols[::-1].index(transient_code)
        
        max_transient_samples = l_idx-f_idx+1-sample_size
        if max_transient_samples > 0:
            num_transient_samples = min(max_samples_per_period, max_transient_samples)
            
            if num_transient_samples == max_transient_samples:
                step_max = 1
            else:
                step_max = (max_transient_samples-1) // (max_samples_per_period-1)
            step_wanted = np.inf
            step = min(step_wanted, step_max)
            
            for idx in range(num_transient_samples):
                f_idx_c = f_idx+idx*step
                l_idx_c = f_idx_c+sample_size
                df_sample = df_vars.iloc[f_idx_c:l_idx_c, :].copy()
                df_sample.insert(loc=0, column='id', value=sample_id)
                df_samples_test = df_samples_test.append(df_sample, ignore_index=True)
                y_test.append(transient_code)
                sample_id += 1
    
    # Processa período em regime - 100% para TESTE
    if class_code in set_ols:
        f_idx = ols.index(class_code)
        if f_idx-(sample_size-1) > 0:
            f_idx = f_idx-(sample_size-1)
        else:
            f_idx = 0
        l_idx = len(ols)-1-ols[::-1].index(class_code)
        if l_idx+(sample_size-1) < len(ols)-1:
            l_idx = l_idx+(sample_size-1)
        else:
            l_idx = len(ols)-1
        
        max_in_regime_samples = l_idx-f_idx+1-sample_size
        if max_in_regime_samples > 0:
            num_in_regime_samples = min(max_samples_per_period, max_in_regime_samples)
            
            if num_in_regime_samples == max_in_regime_samples:
                step_max = 1
            else:
                step_max = (max_in_regime_samples-1) // (max_samples_per_period-1)
            step_wanted = sample_size
            step = min(step_wanted, step_max)
            
            for idx in range(num_in_regime_samples):
                f_idx_c = f_idx+idx*step
                l_idx_c = f_idx_c+sample_size
                df_sample = df_vars.iloc[f_idx_c:l_idx_c, :].copy()
                df_sample.insert(loc=0, column='id', value=sample_id)
                df_samples_test = df_samples_test.append(df_sample, ignore_index=True)
                y_test.append(class_code)
                sample_id += 1
    
    return df_samples_test, y_test

def extract_samples_for_train(df, start_id=0):
    """
    Extrai apenas as amostras NORMAIS de TREINO de um arquivo:
    - 60% das amostras normais
    """
    # Obtém os rótulos das observações
    ols = list(df['class'])
    set_ols = set()
    for ol in ols:
        if ol in set_ols or np.isnan(ol):
            continue
        set_ols.add(int(ol))
    
    # Descarta os rótulos e substitui nan por 0
    df_vars = df.drop('class', axis=1).fillna(0)
    
    # Inicializa objetos que serão retornados
    df_samples_train = pd.DataFrame()
    y_train = []
    sample_id = start_id
    
    # Processa apenas período normal - 60% para TREINO
    if normal_class_code in set_ols:
        f_idx = ols.index(normal_class_code)
        l_idx = len(ols)-1-ols[::-1].index(normal_class_code)
        
        max_samples_normal = l_idx-f_idx+1-sample_size
        if max_samples_normal > 0:
            num_normal_samples = min(max_samples_per_period, max_samples_normal)
            num_train_samples = int(split_range * num_normal_samples)
            
            if num_train_samples > 0:
                if num_normal_samples == max_samples_normal:
                    step_max = 1
                else:
                    step_max = (max_samples_normal-1) // (max_samples_per_period-1)
                step_wanted = sample_size
                step = min(step_wanted, step_max)
                
                for idx in range(num_train_samples):
                    f_idx_c = l_idx-sample_size+1-(num_normal_samples-1-idx)*step
                    l_idx_c = f_idx_c+sample_size
                    df_sample = df_vars.iloc[f_idx_c:l_idx_c, :].copy()
                    df_sample.insert(loc=0, column='id', value=sample_id)
                    df_samples_train = df_samples_train.append(df_sample, ignore_index=True)
                    y_train.append(normal_class_code)
                    sample_id += 1
    
    return df_samples_train, y_train

def train_test_calc_scores(X_train, y_train, X_test, y_test, scores, clfs):
    X_train.reset_index(inplace=True, drop=True)
    X_test.reset_index(inplace=True, drop=True)    
    for clf_name, clf in clfs.items():
        # Treino
        t0 = time()
        clf.fit(X_train, y_train)
        t_train = time() - t0
      # Teste
        t0 = time()
        y_pred = clf.predict(X_test)
        t_test = time() - t0
      # Calcula as metricas de desempenho
        ret = precision_recall_fscore_support(y_test, y_pred, average='micro')
        p, r, f1, _ = ret
        scores = scores.append({'CLASSIFICADOR': clf_name, 
                                'PRECISAO': p,
                                'REVOGACAO': r,
                                'F1': f1,
                                'TREINAMENTO [s]': t_train, 
                                'TESTE [s] ': t_test}, ignore_index=True)  
    return scores


# Gets all selected instances but maintains only those with any type of undesirable event
real_instances = pd.DataFrame(class_and_file_generator(data_path, 
                                                       real=True,
                                                       simulated=False, 
                                                       drawn=False),
                              columns=['class_code', 'instance_path'])
real_instances = real_instances.loc[real_instances.iloc[:,0].isin(abnormal_classes_codes)].reset_index(drop=True)

# Adiciona coluna com número do poço
real_instances['well_number'] = real_instances['instance_path'].apply(extract_well_number)

# For each real instance with any type of undesirable event
scores = pd.DataFrame()
ignored_instances = 0
used_instances = 0

for i, row in real_instances.iterrows():
    # Loads the current instance (arquivo de teste)
    class_code, instance_path, current_well = row
    print(f'Instância {i+1}: {instance_path} (Poço {current_well})')
    df_test = load_instance(instance_path)
    
    # Ignores instances without sufficient normal periods
    normal_period_size = (df_test['class']==float(normal_class_code)).sum()
    if normal_period_size < min_normal_period_size:
        ignored_instances += 1
        print(f'\tignorado porque normal_period_size é insuficiente para treinamento ({normal_period_size})\n')
        continue
    
    # Extrai amostras de TESTE do arquivo atual (40% normal + 100% falha)
    df_samples_test, y_test = extract_samples_for_test(df_test, class_code)
    
    if len(y_test) == 0:
        ignored_instances += 1
        print(f'\tignorado porque não há amostras de teste suficientes\n')
        continue
    
    # Coleta amostras de TREINO de OUTROS ARQUIVOS (exceto do mesmo poço)
    df_samples_train = pd.DataFrame()
    y_train = []
    train_sample_id = 0
    
    print(f'\tColetando amostras de treino de outros poços...')
    for j, other_row in real_instances.iterrows():
        if i == j:  # Pula o arquivo atual
            continue
        
        other_class_code, other_instance_path, other_well = other_row
        
        # NÃO USA arquivos do mesmo poço
        if current_well == other_well:
            print(f'\t\tPulando {other_instance_path.name} (mesmo poço {other_well})')
            continue
        
        # Carrega o outro arquivo
        try:
            df_other = load_instance(other_instance_path)
            
            # Verifica se tem período normal suficiente
            other_normal_period_size = (df_other['class']==float(normal_class_code)).sum()
            if other_normal_period_size < min_normal_period_size:
                continue
            
            # Extrai 60% das amostras normais para treino
            df_other_train, y_other_train = extract_samples_for_train(df_other, start_id=train_sample_id)
            
            if len(y_other_train) > 0:
                df_samples_train = df_samples_train.append(df_other_train, ignore_index=True)
                y_train.extend(y_other_train)
                train_sample_id += len(y_other_train)
                print(f'\t\t+ {len(y_other_train)} amostras de {other_instance_path.name} (Poço {other_well})')
        except Exception as e:
            print(f'\t\tErro ao processar {other_instance_path}: {e}')
            continue
    
    if len(y_train) == 0:
        ignored_instances += 1
        print(f'\tignorado porque não há amostras de treino de outros poços\n')
        continue
    
    used_instances += 1
    print(f'\tTotal de amostras de treino: {len(y_train)}')
    print(f'\tTotal de amostras de teste: {len(y_test)}')
    
    # Changes types of the labels (tsfresh's requirement)
    y_train = np.array(y_train)
    y_test = np.array(y_test)
    
    # We want binary classification: 1 for inliers and -1 for outliers (sklearn's requirement)
    y_train[y_train!=normal_class_code] = -1
    y_train[y_train==normal_class_code] = 1    
    y_test[y_test!=normal_class_code] = -1
    y_test[y_test==normal_class_code] = 1
    
    # Drops the bad vars
    good_vars = np.isnan(df_samples_train[vars]).mean(0) <= max_nan_percent
    std_vars = np.nanstd(df_samples_train[vars], 0)
    good_vars &= (std_vars > std_vars_min)    
    good_vars = list(good_vars.index[good_vars])
    bad_vars = list(set(vars)-set(good_vars))
    df_samples_train.drop(columns=bad_vars, inplace=True, errors='ignore')
    df_samples_test.drop(columns=bad_vars, inplace=True, errors='ignore')
    
    # Normalizes the samples (zero mean and unit variance)
    scaler = preprocessing.StandardScaler()
    df_samples_train[good_vars] = scaler.fit_transform(df_samples_train[good_vars]).astype('float32')
    df_samples_test[good_vars] = scaler.transform(df_samples_test[good_vars]).astype('float32')
    
    # Extracts features from samples
    print(f'\tExtraindo features...')
    X_train = extract_features(df_samples_train, 
                               column_id='id', 
                               column_sort='timestamp', 
                               default_fc_parameters=df_fc_p,
                               impute_function=impute,
                               n_jobs=0,
                               disable_progressbar=disable_progressbar)
    X_train = X_train.reset_index(drop=True)
    X_test = extract_features(df_samples_test, 
                              column_id='id', 
                              column_sort='timestamp',
                              default_fc_parameters=df_fc_p,
                              impute_function=impute,
                              n_jobs=0,
                              disable_progressbar=disable_progressbar)
    X_test = X_test.reset_index(drop=True)
    
    # LISTA DE CLASSIFICADORES A SEREM EXPERIMENTADOS
    clfs = {'Local Outlier Factor': LocalOutlierFactor(n_neighbors=20, algorithm='auto', 
                                               metric='euclidean', contamination="auto", novelty=True),
            'Floresta de Isolamento': IsolationForest(n_estimators=200, max_samples=0.75, contamination=0.1, 
                                                      max_features=1.0, bootstrap=False, 
                                                      random_state=1, verbose=0),
            'Envelope Eliptico MCD': EllipticEnvelope(contamination=0.001, assume_centered=False, 
                               support_fraction=0.99, random_state=1),
            'One Class SVM': OneClassSVM(kernel='sigmoid', gamma=0.01, nu=0.001),
            'One Class SVM (benchmark Vargas)': OneClassSVM(kernel='sigmoid', gamma='scale', nu=0.5),
            'Floresta de Isolamento (benchmark Vargas)': IsolationForest(n_jobs=None, 
                                                   contamination=0.000001, random_state=random_state),
            'Dummy': DummyClassifier(strategy='constant', constant=1)
            }    
    
    # Trains, tests and calculates the scores
    print(f'\tTreinando e testando classificadores...')
    scores = train_test_calc_scores(X_train, y_train, X_test, y_test, scores, clfs)
    print(f'\tConcluído!\n')
    
print(f'Número de instâncias utilizadas: {used_instances}')
print(f'Número de instâncias ignoradas: {ignored_instances}')

print(f'Características utilizadas: {list(df_fc_p.keys())}')

scores.to_csv(r'./results/2-0_anomaly_detection_scores_reais_cross_well.csv')

# Médias (só calcula se houver dados)
if len(scores) > 0:
    mean_score_table = scores.groupby('CLASSIFICADOR').mean().sort_values(by=['F1'], ascending=False)
    mean_score_table.to_csv(r'./results/2-0_anomaly_detection_scores_medias_reais_cross_well.csv')
    display(mean_score_table)
    
    # Desvios Padrão
    std_score_table = scores.groupby('CLASSIFICADOR').std().sort_values(by=['F1'], ascending=True)
    std_score_table.to_csv(r'./results/2-0_anomaly_detection_scores_desvios_padrao_reais_cross_well.csv')
    display(std_score_table)
else:
    print('\nNENHUMA INSTÂNCIA FOI PROCESSADA COM SUCESSO!')
    print('Verifique se há arquivos com períodos normais suficientes.')

# Calcular tempo total do notebook Jupyter
print(f'\nTempo total de execução (hh:mm:ss.ms): {datetime.now() - start_time}')


Instância 1: data\1\WELL-00001_20140124213136.csv (Poço 1)
	ignorado porque normal_period_size é insuficiente para treinamento (959)

Instância 2: data\1\WELL-00002_20140126200050.csv (Poço 2)
	ignorado porque normal_period_size é insuficiente para treinamento (1138)

Instância 3: data\1\WELL-00006_20170801063614.csv (Poço 6)
	Coletando amostras de treino de outros poços...
		Pulando WELL-00006_20170802123000.csv (mesmo poço 6)
		Pulando WELL-00006_20180618060245.csv (mesmo poço 6)
		+ 9 amostras de WELL-00002_20131104014101.csv (Poço 2)
		+ 9 amostras de WELL-00003_20141122214325.csv (Poço 3)
		+ 9 amostras de WELL-00003_20170728150240.csv (Poço 3)
		+ 9 amostras de WELL-00009_20170313160804.csv (Poço 9)
		+ 9 amostras de WELL-00010_20171218200131.csv (Poço 10)
		+ 9 amostras de WELL-00011_20140515110134.csv (Poço 11)
		+ 9 amostras de WELL-00011_20140606230115.csv (Poço 11)
		+ 9 amostras de WELL-00011_20140720120102.csv (Poço 11)
		+ 9 amostras de WELL-00011_20140824000118.csv (Poço

,PRECISAO,REVOGACAO,F1,TREINAMENTO [s],TESTE [s]
CLASSIFICADOR,,,,,
Local Outlier Factor,0.548060,0.548060,0.548060,0.005326,0.001344
One Class SVM (benchmark Vargas),0.367394,0.367394,0.367394,0.005864,0.000727
Floresta de Isolamento,0.297840,0.297840,0.297840,0.272379,0.056289
One Class SVM,0.290234,0.290234,0.290234,0.001635,0.000795
Envelope Eliptico MCD,0.267196,0.267196,0.267196,0.142729,0.002157
Floresta de Isolamento (benchmark Vargas),0.242284,0.242284,0.242284,0.141107,0.028160
Dummy,0.219577,0.219577,0.219577,0.000057,0.000028


,PRECISAO,REVOGACAO,F1,TREINAMENTO [s],TESTE [s]
CLASSIFICADOR,,,,,
Dummy,0.059994,0.059994,0.059994,0.000238,0.000167
Floresta de Isolamento (benchmark Vargas),0.111841,0.111841,0.111841,0.015368,0.005732
Envelope Eliptico MCD,0.169520,0.169520,0.169520,0.067197,0.003333
Floresta de Isolamento,0.198394,0.198394,0.198394,0.018826,0.006571
One Class SVM,0.199438,0.199438,0.199438,0.002278,0.001812
One Class SVM (benchmark Vargas),0.255829,0.255829,0.255829,0.004110,0.001117
Local Outlier Factor,0.285085,0.285085,0.285085,0.004016,0.001827



Tempo total de execução (hh:mm:ss.ms): 0:02:28.639947
